In [1]:
import pandas as pd

In [2]:
from amplpy import AMPL,Environment
ampl = AMPL(Environment('/home/asino/Downloads/ampl_linux-intel64/'))
ampl.setOption("solver","/home/asino/Downloads/ampl_linux-intel64/cplex")

In [3]:
ME="me"

In [4]:
bought=pd.read_csv("auction.csv")
dictOfMyPlayers={}
dictOfPlayersBelongingToOtherBidders={}
for idx,player in bought.iterrows():
    if player["Owner"]==ME:
        dictOfMyPlayers[player["Name"]]=player["Price"]
    else:
        dictOfPlayersBelongingToOtherBidders[player["Name"]]=player["Price"]
print(500-bought.groupby("Owner").sum())
print(bought.groupby("Owner").count())

            Price  Unnamed: 3
Owner                        
bryan          29       500.0
fedeg         350       500.0
fedemiglia      4       500.0
gatti           0         1.0
marco          20       500.0
me             11       500.0
na            492       500.0
            Name  Price  Unnamed: 3
Owner                              
bryan         27     27           0
fedeg          3      3           0
fedemiglia    24     24           0
gatti         23     23           1
marco         28     28           0
me            28     28           0
na             8      8           0


In [5]:
ampl.read("online-auction.mod")
ampl.readData("fantasy-auction-mantra-online.dat")

	online-auction.mod
	line 39 offset 1150
	Caution: 0-dimensional slice


In [ ]:
def suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders):
    plas=ampl.getSet("Players").getValues()
    valuesForOtherPlayersAmpl={}
    valuesForMyPlayersAmpl={}
    for name in plas:
        name=name[0]
        #print(name)
        valuesForMyPlayersAmpl[name]=0
        valuesForOtherPlayersAmpl[name]=0
    budget=500
    for player,cost in dictOfPlayersBelongingToOtherBidders.items():
        valuesForOtherPlayersAmpl[player]=1
    for player,cost in dictOfMyPlayers.items():
        budget-=cost
        valuesForMyPlayersAmpl[player]=1
        
    if valuesForOtherPlayersAmpl:
        ampl.getParameter("alreadyBoughtByOther").setValues(valuesForOtherPlayersAmpl)
    if valuesForMyPlayersAmpl:
        ampl.getParameter("alreadyBoughtByMe").setValues(valuesForMyPlayersAmpl)
        
        
    
    ampl.getParameter("budget").set(budget)
    ampl.solve()


In [ ]:
suggestTeam(dictOfMyPlayers,dictOfPlayersBelongingToOtherBidders)

In [ ]:
buy=ampl.getVariable("putPlayerInSlot")
cost=ampl.getParameter("cost")
pfs=ampl.getParameter("performance")
moduleChosen=ampl.getVariable("bidderChoosesModule")

roles=ampl.getSet("CompatibilityPlayerRole").getValues().toDict()

solution=buy.getValues().toDict()
costs=cost.getValues().toDict()
pfs=pfs.getValues().toDict()
modules=moduleChosen.getValues().toDict()
moduleUsed="AAA"
for moduleName,taken in modules.items():
    if taken>0.5:
        print("Module: {}\n\n".format(moduleName))
        moduleUsed=moduleName

myPls=[]
for key,val in solution.items():
    if key[0]==moduleName:
        if val>0.5:
            myPls.append(key[1])
rolesA={}

for key,val in roles.items():
    if key[0] in myPls:
        if key[0] not in rolesA.keys():
            rolesA[key[0]]=[]
        rolesA[key[0]].append(key[1])
print(rolesA)

In [ ]:
for player in myPls:
    message="{}                 {}   {}  ".format(player,costs[player],rolesA[player])
    print(message)